<a href="https://colab.research.google.com/github/talhaanwarch/OffenseEval2020/blob/master/submissions/English_Crossval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install focal-loss
!pip install keras-tcn==2.8.3

In [0]:
#Setup Kaggle DIR; copy json file ; chmod
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

kaggle.json


In [0]:
#Install kaggle packages
!pip install -q kaggle
!pip install -q kaggle-cli

     |████████████████████████████████| 81kB 3.0MB/s 
     |████████████████████████████████| 5.3MB 8.1MB/s 
     |████████████████████████████████| 112kB 72.7MB/s 
     |████████████████████████████████| 51kB 9.4MB/s 
     |████████████████████████████████| 112kB 75.3MB/s 


In [0]:
#List Kaggle DataSets
#!kaggle datasets list

#List Kaggle Competitions

!kaggle competitions list

ref                                                                deadline             category             reward  teamCount  userHasEntered  
-----------------------------------------------------------------  -------------------  ---------------  ----------  ---------  --------------  
digit-recognizer                                                   2030-01-01 00:00:00  Getting Started   Knowledge       2357           False  
titanic                                                            2030-01-01 00:00:00  Getting Started   Knowledge      16555           False  
house-prices-advanced-regression-techniques                        2030-01-01 00:00:00  Getting Started   Knowledge       4702           False  
connectx                                                           2030-01-01 00:00:00  Getting Started   Knowledge        478           False  
competitive-data-science-predict-future-sales                      2020-12-31 23:59:00  Playground            Kudos       5839    

In [0]:
!kaggle competitions list -s nlp-getting-started


ref                             deadline             category          reward  teamCount  userHasEntered  
------------------------------  -------------------  ---------------  -------  ---------  --------------  
nlp-getting-started             2020-03-23 23:59:00  Getting Started  $10,000       3416            True  
getting-started                 2012-02-26 00:00:00  Featured         $10,000          0           False  
acm-sf-chapter-hackathon-small  2012-09-30 01:00:00  Research            $600         96           False  


In [0]:
!kaggle competitions download -c nlp-getting-started

  0% 0.00/22.2k [00:00<?, ?B/s]
100% 22.2k/22.2k [00:00<00:00, 8.51MB/s]
  0% 0.00/965k [00:00<?, ?B/s]
100% 965k/965k [00:00<00:00, 64.3MB/s]
  0% 0.00/411k [00:00<?, ?B/s]
100% 411k/411k [00:00<00:00, 136MB/s]


# Import lib

In [0]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential,Model
from keras.layers import Embedding,CuDNNGRU,CuDNNLSTM,Dense,Dropout,Bidirectional,BatchNormalization,GlobalMaxPooling1D,Flatten, GlobalAveragePooling1D, MaxPooling1D,SpatialDropout1D,Input,Activation,concatenate,Conv1D
from keras.optimizers import RMSprop,Adam,Adadelta
from keras.initializers import Constant
from sklearn.utils import class_weight
from sklearn.metrics import classification_report,f1_score
import keras
import collections
import numpy as np
from focal_loss import BinaryFocalLoss
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from keras import backend as K
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('stopwords')

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)


Using TensorFlow backend.


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Load data

In [0]:
lemmatizer = WordNetLemmatizer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
stop = stopwords.words('english')


In [0]:
train=pd.read_csv( '/content/train.csv')
train.head()


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [0]:

train['text']=train['text'].str.replace('\d+', '')
train["text"]= train["text"].str.lower()
train["text"] = train["text"].str.replace('[^\w\s]','')
train["text"] = train["text"].apply(lambda x : [lemmatizer.lemmatize(y) for y in w_tokenizer.tokenize(x)])
train["text"] = train["text"].apply(lambda x: [item for item in x if item not in stop])
train["text"] = train["text"].apply(lambda x : " ".join(x))
train.head()

KeyError: ignored

In [0]:
test=pd.read_csv( '/content/test.csv')
test['text']=test['text'].str.replace('\d+', '')
test["text"]= test["text"].str.lower()
test["text"] = test["text"].str.replace('[^\w\s]','')
test["text"] = test["text"].apply(lambda x : [lemmatizer.lemmatize(y) for y in w_tokenizer.tokenize(x)])
test["text"] = test["text"].apply(lambda x: [item for item in x if item not in stop])
test["text"] = test["text"].apply(lambda x : " ".join(x))
test=test["text"] 
test.head()

#Label Encoding

In [0]:
labels=train['target']
train=train['text']

In [0]:
le=LabelEncoder()
labels=le.fit_transform(labels)
print(len(labels))

7613


# Common Parameters

In [0]:
import numpy as np
class_weights = class_weight.compute_class_weight('balanced',np.unique(labels),labels)
class_weights=dict(enumerate(class_weights))

## Cylic learning rate

In [0]:
# https://www.kaggle.com/hireme/fun-api-keras-f1-metric-cyclical-learning-rate/code
from keras.callbacks import Callback
class CyclicLR(Callback):
    """This callback implements a cyclical learning rate policy (CLR).
    The method cycles the learning rate between two boundaries with
    some constant frequency, as detailed in this paper (https://arxiv.org/abs/1506.01186).
    The amplitude of the cycle can be scaled on a per-iteration or 
    per-cycle basis.
    This class has three built-in policies, as put forth in the paper.
    "triangular":
        A basic triangular cycle w/ no amplitude scaling.
    "triangular2":
        A basic triangular cycle that scales initial amplitude by half each cycle.
    "exp_range":
        A cycle that scales initial amplitude by gamma**(cycle iterations) at each 
        cycle iteration.
    For more detail, please see paper.
    
    # Example
        ```python
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., mode='triangular')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```
    
    Class also supports custom scaling functions:
        ```python
            clr_fn = lambda x: 0.5*(1+np.sin(x*np.pi/2.))
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., scale_fn=clr_fn,
                                scale_mode='cycle')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```    
    # Arguments
        base_lr: initial learning rate which is the
            lower boundary in the cycle.
        max_lr: upper boundary in the cycle. Functionally,
            it defines the cycle amplitude (max_lr - base_lr).
            The lr at any cycle is the sum of base_lr
            and some scaling of the amplitude; therefore 
            max_lr may not actually be reached depending on
            scaling function.
        step_size: number of training iterations per
            half cycle. Authors suggest setting step_size
            2-8 x training iterations in epoch.
        mode: one of {triangular, triangular2, exp_range}.
            Default 'triangular'.
            Values correspond to policies detailed above.
            If scale_fn is not None, this argument is ignored.
        gamma: constant in 'exp_range' scaling function:
            gamma**(cycle iterations)
        scale_fn: Custom scaling policy defined by a single
            argument lambda function, where 
            0 <= scale_fn(x) <= 1 for all x >= 0.
            mode paramater is ignored 
        scale_mode: {'cycle', 'iterations'}.
            Defines whether scale_fn is evaluated on 
            cycle number or cycle iterations (training
            iterations since start of cycle). Default is 'cycle'.
    """

    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())

In [0]:
clr = CyclicLR(base_lr=0.001, max_lr=0.005,
                        step_size=4., mode='exp_range',
                        gamma=0.99994)

# Tokenize data

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
max_words = 10000 #frequency of words to be kept
max_len = 200

tokenize = Tokenizer(num_words=max_words)
tokenize.fit_on_texts(train)
sequences = tokenize.texts_to_sequences(train)
word_index = tokenize.word_index
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len,padding='post')

In [0]:
len(sequences_matrix),len(labels)

(7613, 7613)

In [0]:
num_words = min(max_words, len(word_index)) + 1
print(num_words)

10001


In [0]:
test_sequences = tokenize.texts_to_sequences(test)
X_test_sequences = sequence.pad_sequences(test_sequences,maxlen=max_len,padding='post')

In [0]:
len(X_test_sequences)

3263

# Embeddings

In [0]:
embedding_path1 = "drive/My Drive/dataset/OLID/wiki-news-300d-1M.vec"
embedding_path2 = "drive/My Drive/dataset/OLID/glove/glove.840B.300d.txt"
embed_size = 300

In [0]:
def get_coefs(word,*arr):
    return word, np.asarray(arr, dtype='float32')

def build_matrix(embedding_path, word_index):
    embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path))

    nb_words = min(max_words, len(word_index))
    embedding_matrix = np.zeros((nb_words + 1, embed_size))
    for word, i in word_index.items():
        if i >= max_words:
            continue
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [0]:
fasttext=build_matrix(embedding_path1, word_index)
glove_emb=build_matrix(embedding_path2, word_index)

In [0]:
embeddings=np.mean((fasttext,glove_emb),axis=0)
embeddings.shape

(10001, 300)

# Base Models

In [0]:
train.shape,labels.shape

((7613,), (7613,))

## Count Vectorizer

In [0]:

def logistic_param_selection(X, y, func,nfolds):
    C= [7,8,9,10,12,15,20,25]
    param_grid = {'C': C}
    grid_search = GridSearchCV(make_pipeline(func, LogisticRegression(solver='lbfgs',max_iter=500,class_weight=class_weights)),
                    param_grid={'logisticregression__C':C}, cv=nfolds,scoring='f1_macro')
    grid_search.fit(X, y)
    return grid_search.best_score_,grid_search.best_params_



In [0]:
cv_score,c=logistic_param_selection(train,labels,CountVectorizer(),5)
cv_score

0.6789070385269321

In [0]:
count_vectorizer = CountVectorizer()
count_vectorizer.fit(train)
X_train_cv = count_vectorizer.transform(train)
X_test_cv=  count_vectorizer.transform(test)

In [0]:
cv_classifier = LogisticRegression(solver='lbfgs',C=c['logisticregression__C'],max_iter=500,class_weight=class_weights)
cv_classifier.fit(X_train_cv, labels)

LogisticRegression(C=7,
                   class_weight={0: 0.8766697374481806, 1: 1.1637114032405993},
                   dual=False, fit_intercept=True, intercept_scaling=1,
                   l1_ratio=None, max_iter=500, multi_class='auto', n_jobs=None,
                   penalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
                   verbose=0, warm_start=False)

In [0]:
cv_test=cv_classifier.predict_proba(X_test_cv)[:,1]

## TF IDF word vectorizer

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 4),
    max_features=10000)
word_vectorizer.fit(pd.concat([train, test]))
train_word_features = word_vectorizer.transform(train)
test_word_features = word_vectorizer.transform(test)

In [0]:
tfw_score,c=logistic_param_selection(train,labels,word_vectorizer,5)
tfw_score

0.6814031008661787

In [0]:
tfw_classifier = LogisticRegression(solver='lbfgs',C=c['logisticregression__C'],max_iter=500)
tfw_classifier.fit(train_word_features, labels)


LogisticRegression(C=7, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
tfw_test = tfw_classifier.predict_proba(test_word_features)[:,1]

## TF IDF char vectorizer

In [0]:
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(1, 6),
    max_features=30000)
char_vectorizer.fit(train)
train_char_features = char_vectorizer.transform(train)
test_char_features = char_vectorizer.transform(test)

In [0]:
tfc_score,c=logistic_param_selection(train,labels,char_vectorizer,5)
tfc_score

0.6909844962829798

In [0]:
tfc_classifier = LogisticRegression(solver='lbfgs',C=c['logisticregression__C'],max_iter=500)
tfc_classifier.fit(train_char_features, labels)


LogisticRegression(C=7, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
tfc_test = tfc_classifier.predict_proba(test_char_features)[:,1]

# Models

##GRU

In [0]:
def model_gru():
  inp = Input(shape=(max_len,))
  x = Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False)(inp)
  x = SpatialDropout1D(0.1)(x)
  x = Bidirectional(CuDNNLSTM(50, return_sequences=True))(x)
  x, x_h, x_c = Bidirectional(CuDNNGRU(50, return_sequences=True, return_state = True))(x)
  avg_pool = GlobalAveragePooling1D()(x)
  max_pool = GlobalMaxPooling1D()(x)
  conc = concatenate([avg_pool, x_h, max_pool])
  outp = Dense(1, activation="sigmoid")(conc)    
  model = Model(inputs=inp, outputs=outp)
  model.compile(loss=BinaryFocalLoss(gamma=2), optimizer='adam', metrics=['accuracy'])
  return model

## GRU attention

In [0]:
#https://github.com/zake7749/DeepToxic
from keras.layers import Layer,Lambda
from keras import initializers
from keras.engine import InputSpec, Layer
from keras import backend as K

class AttentionWeightedAverage(Layer):
    """
    Computes a weighted average of the different channels across timesteps.
    Uses 1 parameter pr. channel to compute the attention value for a single timestep.
    """

    def __init__(self, return_attention=False, **kwargs):
        self.init = initializers.get('uniform')
        self.supports_masking = True
        self.return_attention = return_attention
        super(AttentionWeightedAverage, self).__init__(** kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(ndim=3)]
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[2], 1),
                                 name='{}_W'.format(self.name),
                                 initializer=self.init)
        self.trainable_weights = [self.W]
        super(AttentionWeightedAverage, self).build(input_shape)

    def call(self, x, mask=None):
        # computes a probability distribution over the timesteps
        # uses 'max trick' for numerical stability
        # reshape is done to avoid issue with Tensorflow
        # and 1-dimensional weights
        logits = K.dot(x, self.W)
        x_shape = K.shape(x)
        logits = K.reshape(logits, (x_shape[0], x_shape[1]))
        ai = K.exp(logits - K.max(logits, axis=-1, keepdims=True))

        # masked timesteps have zero weight
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            ai = ai * mask
        att_weights = ai / (K.sum(ai, axis=1, keepdims=True) + K.epsilon())
        weighted_input = x * K.expand_dims(att_weights)
        result = K.sum(weighted_input, axis=1)
        if self.return_attention:
            return [result, att_weights]
        return result

    def get_output_shape_for(self, input_shape):
        return self.compute_output_shape(input_shape)

    def compute_output_shape(self, input_shape):
        output_len = input_shape[2]
        if self.return_attention:
            return [(input_shape[0], output_len), (input_shape[0], input_shape[1])]
        return (input_shape[0], output_len)

    def compute_mask(self, input, input_mask=None):
        if isinstance(input_mask, list):
            return [None] * len(input_mask)
        else:
            return None

In [0]:
def model_gru_attn():
  inp = Input(shape=(max_len,))
  x = Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False)(inp)
  x = SpatialDropout1D(0.1)(x)
  x = Bidirectional(CuDNNLSTM(50, return_sequences=True))(x)
  x = Bidirectional(CuDNNGRU(50, return_sequences=True))(x)
  avg_pool = GlobalAveragePooling1D()(x)
  max_pool = GlobalMaxPooling1D()(x)
  last = Lambda(lambda t: t[:, -1])(x)
  attn = AttentionWeightedAverage()(x)
  conc = concatenate([avg_pool,  max_pool,last,attn])
  outp = Dense(1, activation="sigmoid")(conc)    
  model = Model(inputs=inp, outputs=outp)
  model.compile(loss=BinaryFocalLoss(gamma=2), optimizer='adam', metrics=['accuracy'])
  return model

##TCN

In [0]:
from tcn import TCN

def wave_net_activation(x):
    # type: (Layer) -> Layer
    """This method defines the activation used for WaveNet
    described in https://deepmind.com/blog/wavenet-generative-model-raw-audio/
    Args:
        x: The layer we want to apply the activation to
    Returns:
        A new layer with the wavenet activation applied
    """
    tanh_out = Activation('tanh')(x)
    sigm_out = Activation('sigmoid')(x)
    return keras.layers.multiply([tanh_out, sigm_out])

def model_tcn(embedding_matrix):
    
    inp = Input(shape=(max_len,))
    x = Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False)(inp)
    x = SpatialDropout1D(0.1)(x)
    x = TCN(128,dilations = [1, 2, 4], return_sequences=True ,name = 'tnc1')(x)
    x = wave_net_activation(x)
    x = TCN(64,dilations = [1, 2, 4], return_sequences=True, name = 'tnc2')(x)
    x = wave_net_activation(x)
    #x = TCN(32,dilations = [1, 2, 4], return_sequences=True, activation = 'wavenet',name = 'tnc3')(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(64, activation="relu")(conc)
    conc = Dense(32, activation="relu")(conc)

    conc = Dropout(0.1)(conc)
    outp = Dense(1, activation="sigmoid")(conc)    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss=BinaryFocalLoss(gamma=2), optimizer='adam', metrics=['accuracy'])
    
    return model

##KIM

In [0]:
conv_filters=128
def model_kim():
  inp = Input(shape=(max_len,))
  emb = Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False)(inp)
  # Specify each convolution layer and their kernel siz i.e. n-grams 
  conv1_1 = Conv1D(filters=conv_filters, kernel_size=3)(emb)
  btch1_1 = BatchNormalization()(conv1_1)
  drp1_1  = Dropout(0.2)(btch1_1)
  actv1_1 = Activation('relu')(drp1_1)
  glmp1_1 = GlobalMaxPooling1D()(actv1_1)

  conv1_2 = Conv1D(filters=conv_filters, kernel_size=4)(emb)
  btch1_2 = BatchNormalization()(conv1_2)
  drp1_2  = Dropout(0.2)(btch1_2)
  actv1_2 = Activation('relu')(drp1_2)
  glmp1_2 = GlobalMaxPooling1D()(actv1_2)

  conv1_3 = Conv1D(filters=conv_filters, kernel_size=5)(emb)
  btch1_3 = BatchNormalization()(conv1_3)
  drp1_3  = Dropout(0.2)(btch1_3)
  actv1_3 = Activation('relu')(drp1_3)
  glmp1_3 = GlobalMaxPooling1D()(actv1_3)

  conv1_4 = Conv1D(filters=conv_filters, kernel_size=6)(emb)
  btch1_4 = BatchNormalization()(conv1_4)
  drp1_4  = Dropout(0.2)(btch1_4)
  actv1_4 = Activation('relu')(drp1_4)
  glmp1_4 = GlobalMaxPooling1D()(actv1_4)

  # Gather all convolution layers
  cnct = concatenate([glmp1_1, glmp1_2, glmp1_3, glmp1_4], axis=1)
  drp1 = Dropout(0.2)(cnct)

  dns1  = Dense(32, activation='relu')(drp1)
  btch1 = BatchNormalization()(dns1)
  drp2  = Dropout(0.2)(btch1)

  out = Dense(1, activation='sigmoid')(drp2)   
  model = Model(inputs=inp, outputs=out)
  model.compile(loss=BinaryFocalLoss(gamma=2), optimizer='adam', metrics=['accuracy'])
  return model


# Train model

##GRU

In [0]:
kfold = StratifiedKFold(n_splits=5, shuffle=False, random_state=2020)
gru_scores = []
gru_test=[]
for train, val in kfold.split(sequences_matrix, labels):
  gru_model=model_gru()
  
  gru_model.fit(sequences_matrix[train], labels[train],batch_size=64,epochs=10,verbose=2,
            validation_data=(sequences_matrix[val],labels[val]),callbacks=[clr,])
  y_pred = gru_model.predict(sequences_matrix[val], batch_size=64, verbose=1)

  gru_test.append(gru_model.predict(X_test_sequences, batch_size=64, verbose=1).ravel())

  y_pred = (y_pred > 0.5)
  f1=f1_score(labels[val], y_pred, average='macro')
  gru_scores.append(f1)
  keras.backend.clear_session()

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Train on 6090 samples, validate on 1523 samples
Epoch 1/10
 - 11s - loss: 0.1270 - acc: 0.7690 - val_loss: 0.1402 - val_acc: 0.7295
Epoch 2/10
 - 3s - loss: 0.1103 - acc: 0.8136 - val_loss: 0.1270 - val_acc: 0.7800
Epoch 3/10
 - 3s - loss: 0.1067 - acc: 0.8233 - val_loss: 0.1372 - val_acc: 0.7748
Epoch 4/10
 - 3s - loss: 0.0951 - acc: 0.8407 - val_loss: 0.1250 - val_acc: 0.7899
Epoch 5/10
 - 3s - loss: 0.0869 - acc: 0.8517 - val_loss: 0.1531 - val_acc: 0.7485
Epoch 6/10
 - 3s - loss: 0.0765 - acc: 0.8750 - val_loss: 0.1431 - val_acc: 0.7584
Epoch 7/10
 - 3s - loss: 0.0687 - acc: 0.8880 - val_loss: 0.1539 - val_acc: 0.7748
Epoch 8/10
 - 3s - loss: 0.0612 - acc: 0.9053 - val_loss: 0.1786 - val_acc: 0.7643
Epoch 9/10
 - 3s - loss: 0.0526 - acc: 0.9182 - val_loss: 0.1840 - val_acc: 0.7617
Epoch 10/10
 - 3s - loss: 0.0464 - acc: 0.9299 - val_loss: 0.2406 - val_acc: 0.7610
3263/3263 [==============================] - 1s 218us/step
Train on 6090 samples, validate on 1523 samples
Epoch 1/10
 -

In [0]:
len(gru_test),gru_test[0].shape,gru_scores

(5,
 (3263,),
 [0.738907727666629,
  0.6714840379637619,
  0.7020580791030875,
  0.7051829957967655,
  0.7869304524683695])

In [0]:
np.mean(np.array(gru_test),axis=0).shape

(3263,)

## GRU attention

In [0]:
kfold = StratifiedKFold(n_splits=5, shuffle=False, random_state=2020)
atten_scores = []
atten_test=[]
for train, val in kfold.split(sequences_matrix, labels):
  attn_gru_model=model_gru_attn()
  
  attn_gru_model.fit(sequences_matrix[train], labels[train],batch_size=64,epochs=10,verbose=2,
            validation_data=(sequences_matrix[val],labels[val]),callbacks=[clr,])
  y_pred = attn_gru_model.predict(sequences_matrix[val], batch_size=64, verbose=1)

  atten_test.append(attn_gru_model.predict(X_test_sequences, batch_size=64, verbose=1).ravel())

  y_pred = (y_pred > 0.5)
  f1=f1_score(labels[val], y_pred, average='macro')
  atten_scores.append(f1)
  keras.backend.clear_session()

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Train on 6090 samples, validate on 1523 samples
Epoch 1/10
 - 4s - loss: 0.1275 - acc: 0.7680 - val_loss: 0.1218 - val_acc: 0.7932
Epoch 2/10
 - 4s - loss: 0.1101 - acc: 0.8131 - val_loss: 0.1285 - val_acc: 0.7859
Epoch 3/10
 - 3s - loss: 0.1027 - acc: 0.8287 - val_loss: 0.1249 - val_acc: 0.7965
Epoch 4/10
 - 3s - loss: 0.0964 - acc: 0.8383 - val_loss: 0.1274 - val_acc: 0.7853
Epoch 5/10
 - 3s - loss: 0.0903 - acc: 0.8527 - val_loss: 0.1315 - val_acc: 0.7715
Epoch 6/10
 - 3s - loss: 0.0789 - acc: 0.8693 - val_loss: 0.1662 - val_acc: 0.7636
Epoch 7/10
 - 3s - loss: 0.0726 - acc: 0.8821 - val_loss: 0.1580 - val_acc: 0.7577
Epoch 8/10
 - 3s - loss: 0.0648 - acc: 0.8969 - val_loss: 0.1758 - val_acc: 0.7643
Epoch 9/10
 - 3s - loss: 0.0555 - acc: 0.9125 - val_loss: 0.1817 - val_acc: 0.7682
Epoch 10/10
 - 3s - loss: 0.0464 - acc: 0.9315 - val_loss: 0.2008 - val_acc: 0.7814
3263/3263 [==============================] - 1s 217us/step
Train on 6090 samples, validate on 1523 samples
Epoch 1/10
 - 

In [0]:
atten_scores 


[0.7705815856030116,
 0.6561528770341645,
 0.6828430812478308,
 0.7113317542586943,
 0.7709183253788654]

##TCN

In [0]:
kfold = StratifiedKFold(n_splits=5, shuffle=False)
tcn_scores = []
tcn_test=[]
for train, val in kfold.split(sequences_matrix, labels):
  model=model_tcn(embeddings)
  
  model.fit(sequences_matrix[train], labels[train],batch_size=64,epochs=10,verbose=2,
            validation_data=(sequences_matrix[val],labels[val]),callbacks=[clr,])
  y_pred = model.predict(sequences_matrix[val], batch_size=64, verbose=1)

  tcn_test.append(model.predict(X_test_sequences, batch_size=64, verbose=1).ravel())

  y_pred = (y_pred > 0.5)
  f1=f1_score(labels[val], y_pred, average='macro')
  tcn_scores.append(f1)
  keras.backend.clear_session()

Train on 6090 samples, validate on 1523 samples
Epoch 1/10
 - 7s - loss: 0.1506 - acc: 0.6734 - val_loss: 0.1387 - val_acc: 0.7433
Epoch 2/10
 - 3s - loss: 0.1262 - acc: 0.7785 - val_loss: 0.1324 - val_acc: 0.7695
Epoch 3/10
 - 3s - loss: 0.1191 - acc: 0.8023 - val_loss: 0.1439 - val_acc: 0.7216
Epoch 4/10
 - 3s - loss: 0.1190 - acc: 0.7995 - val_loss: 0.1335 - val_acc: 0.7433
Epoch 5/10
 - 3s - loss: 0.1130 - acc: 0.8153 - val_loss: 0.1286 - val_acc: 0.7617
Epoch 6/10
 - 3s - loss: 0.1091 - acc: 0.8238 - val_loss: 0.1246 - val_acc: 0.7643
Epoch 7/10
 - 3s - loss: 0.1084 - acc: 0.8271 - val_loss: 0.1304 - val_acc: 0.7584
Epoch 8/10
 - 3s - loss: 0.1036 - acc: 0.8355 - val_loss: 0.1494 - val_acc: 0.7479
Epoch 9/10
 - 3s - loss: 0.0963 - acc: 0.8486 - val_loss: 0.1398 - val_acc: 0.7505
Epoch 10/10
 - 3s - loss: 0.0966 - acc: 0.8504 - val_loss: 0.1340 - val_acc: 0.7722
3263/3263 [==============================] - 1s 219us/step
Train on 6090 samples, validate on 1523 samples
Epoch 1/10
 - 

##KIM

In [0]:
kfold = StratifiedKFold(n_splits=5, shuffle=False)
kim_scores = []
kim_test=[]
for train, val in kfold.split(sequences_matrix, labels):
  model=model_kim()
  
  model.fit(sequences_matrix[train], labels[train],batch_size=64,epochs=10,verbose=2,
            validation_data=(sequences_matrix[val],labels[val]),callbacks=[clr,])
  y_pred = model.predict(sequences_matrix[val], batch_size=64, verbose=1)

  kim_test.append(model.predict(X_test_sequences, batch_size=64, verbose=1).ravel())

  y_pred = (y_pred > 0.5)
  f1=f1_score(labels[val], y_pred, average='macro')
  kim_scores.append(f1)
  keras.backend.clear_session()

Train on 6090 samples, validate on 1523 samples
Epoch 1/10
 - 4s - loss: 0.1708 - acc: 0.7342 - val_loss: 0.1666 - val_acc: 0.7249
Epoch 2/10
 - 2s - loss: 0.1156 - acc: 0.8067 - val_loss: 0.1269 - val_acc: 0.7827
Epoch 3/10
 - 2s - loss: 0.0943 - acc: 0.8440 - val_loss: 0.1302 - val_acc: 0.7617
Epoch 4/10
 - 2s - loss: 0.0830 - acc: 0.8663 - val_loss: 0.1466 - val_acc: 0.7610
Epoch 5/10
 - 2s - loss: 0.0722 - acc: 0.8954 - val_loss: 0.1527 - val_acc: 0.7413
Epoch 6/10
 - 2s - loss: 0.0636 - acc: 0.9128 - val_loss: 0.1798 - val_acc: 0.7374
Epoch 7/10
 - 2s - loss: 0.0539 - acc: 0.9289 - val_loss: 0.1951 - val_acc: 0.7111
Epoch 8/10
 - 2s - loss: 0.0488 - acc: 0.9348 - val_loss: 0.1653 - val_acc: 0.7643
Epoch 9/10
 - 2s - loss: 0.0426 - acc: 0.9415 - val_loss: 0.1757 - val_acc: 0.7636
Epoch 10/10
 - 2s - loss: 0.0388 - acc: 0.9471 - val_loss: 0.2623 - val_acc: 0.7466
3263/3263 [==============================] - 0s 122us/step
Train on 6090 samples, validate on 1523 samples
Epoch 1/10
 - 

#**Result**

# Result

In [0]:
gru_scores=np.average(np.array(gru_scores))
atten_scores=np.average(np.array(atten_scores))
tcn_scores=np.average(np.array(tcn_scores))
kim_scores=np.average(np.array(kim_scores))

In [0]:
print('f1 score of count vec' ,cv_score)
print('f1 score of word tfidf' ,tfw_score)
print('f1 score of char tfidf' ,tfc_score)

print('f1 score of RNN' ,gru_scores)
print('f1 score of gated attention',atten_scores )
print('f1 score of tcn',tcn_scores)
print('f1 score of kim',kim_scores)


f1 score of count vec 0.6789070385269321
f1 score of word tfidf 0.6814031008661787
f1 score of char tfidf 0.6909844962829798
f1 score of RNN 0.7209126585997228
f1 score of gated attention 0.7183655247045133
f1 score of tcn 0.7401977975331973
f1 score of kim 0.720860230243704


In [0]:
kim_test_avg=np.mean(np.array(kim_test),axis=0)
gru_test_avg=np.mean(np.array(gru_test),axis=0)
atten_test_avg=np.mean(np.array(atten_test),axis=0)
tcn_test_avg=np.mean(np.array(tcn_test),axis=0)


In [0]:
kim_test[0].shape,cv_test.shape,kim_test_avg.shape

((3263,), (3263,), (3263,))

In [0]:
print(cv_test[5:15].round())
print(tfw_test[5:15].round())
print(tfc_test[5:15].round())
print(gru_test_avg[5:15].round())
print(atten_test_avg[5:15].round())
print(kim_test_avg[5:15].round())
print(tcn_test_avg[5:15].round())


[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


# Ensemble

we will choose only those, having f1 greater than 0.7. Once they are selected, we will decode labels, and keep label according to mod

In [0]:
wt=gru_scores+atten_scores+cv_score+tfw_score+tfc_score+tcn_scores+kim_scores
wt

4.9516308467572285

In [0]:
#i am bit changing the distribution in order to give more weightage to ml than dl
y_test=gru_scores/wt * gru_test_avg + atten_scores/wt * atten_test_avg + cv_score/wt * cv_test + tfw_score/wt *tfw_test + tfc_score/wt *tfc_test +kim_scores/wt * kim_test_avg + tcn_scores/wt *tcn_test_avg

In [0]:
y_test[5:15].round()

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [0]:
def decode(y_test):
  y_test[y_test>0.5]=1
  y_test[y_test<0.5]=0
  y_test=y_test.astype('int16').ravel()

  y_test=le.inverse_transform(y_test)
  y_test=pd.DataFrame(y_test,columns=['label'])
  y_test=pd.concat([ids, y_test['label']], axis=1)
  return y_test

In [0]:
y_test[y_test>0.5]=1
y_test[y_test<0.5]=0
y_test=y_test.astype('int16').ravel()

y_test=pd.DataFrame(y_test,columns=['label'])
y_test=pd.concat([ids, y_test['label']], axis=1)
y_test.head()

NameError: ignored

In [0]:
y_test.head()

,label
0,1
1,1
2,1
3,1
4,1


# Submit file

In [0]:
y_test.to_csv('/content/disaster.csv',index=False,header=None)


# Reset

In [0]:
# import keras
# keras.backend.clear_session()